In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.base import BaseEstimator
import optuna
import warnings
warnings.filterwarnings('ignore')

In [2]:
scaler = StandardScaler()
df = pd.read_csv("Hepatitis_C.csv")
X = df.drop(columns='label').to_numpy()
y = df.iloc[:,-1].to_numpy()
print("Data frame shape:",df.shape,"\nFeatures shape:",X.shape,"\nLabels shape",y.shape)
df.head()
X

Data frame shape: (204, 13) 
Features shape: (204, 12) 
Labels shape (204,)


array([[32. ,  0. , 43.2, ..., 80. , 33.8, 75.7],
       [45. ,  0. , 41.7, ..., 71. , 67.4, 70.3],
       [55. ,  0. , 41.5, ..., 80. , 12.4, 69.9],
       ...,
       [64. ,  1. , 29. , ..., 66.7, 64.2, 82. ],
       [46. ,  1. , 33. , ..., 52. , 50. , 71. ],
       [59. ,  1. , 36. , ..., 67. , 34. , 68. ]])

In [4]:
X[:,[0,2,3,4,5,6,7,8,9,11]] = scaler.fit_transform(X[:,[0,2,3,4,5,6,7,8,9,11]])

In [5]:
lr = LogisticRegression()
gnb = GaussianNB()
knn = KNeighborsClassifier()
lda = LinearDiscriminantAnalysis()
svm = SVC()

In [27]:
for model in [gnb, knn, lda, svm]:
    params = model.get_params()
    print(f"{model} has {len(params)} tunable parameters:\n {params}\n")

GaussianNB() has 2 tunable parameters:
 {'priors': None, 'var_smoothing': 1e-09}

KNeighborsClassifier() has 8 tunable parameters:
 {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}

LinearDiscriminantAnalysis() has 7 tunable parameters:
 {'covariance_estimator': None, 'n_components': None, 'priors': None, 'shrinkage': None, 'solver': 'svd', 'store_covariance': False, 'tol': 0.0001}

SVC() has 15 tunable parameters:
 {'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}



In [288]:
class BinaryClassifierNCV(BaseEstimator):
  """Class to tune the hyperparameters of a classifier using optuna and nested cross validation,
  the specified inner cross validation folds are used for hyperparameter tuning and the specified outer folds are used to evaluate the model performance.
  """
  def __init__(self,classifier_name, seed = 42, outer_folds= 5, inner_folds = 3, shuffle = True, trial_num = 10, verbose = True):
    self.shuffle = shuffle
    self.outer_folds = outer_folds
    self.inner_folds = inner_folds
    self.trial_num = trial_num
    self.classifier_name = classifier_name
    self.model = model
    self.shuffle
    self.verbose = verbose
    # Set the random seed for generating the seed for each trial
    np.random.seed(seed)
    self.trials_seeds = np.random.randint(0, 1000, size = trial_num)
    if self.verbose:
      print("The random seeds for each trial are:", self.trials_seeds,"\n")

  # Fit the selected classifier in 10 cross validation trials for hyperparameter tuning
  def fit(self, X, y):
    #Initializing Naive Bayes classifier to be used as a baseline model
    baseline_model = GaussianNB()
    # Initialize an array to store the best parameters for outer cross validation loop
    parameters = []
    #Initializing the dataframes to store the average scores per trial
    scores_df = pd.DataFrame(columns = ["F1", "Balanced Accuracy", "Precision", "MCC", "Recall", "ROC AUC"], index=[f"Trial {i + 1}" for i in range(self.trial_num)])
    baseline_df = pd.DataFrame(columns = ["F1", "Balanced Accuracy", "Precision", "MCC", "Recall", "ROC AUC"], index=[f"Trial {i + 1}" for i in range(self.trial_num)])
  
    ############################# Nested cross validation for hyperparameter tuning ###################################################
    #objective function to be optimized by optuna for each inner cross validation loop
    # This function is called inside the for loop of the outer cross validation loop
    def objective(trial):
      #Knearst neighbors tuning
      if self.classifier_name == "KNeighborsclassifier":
        n_neighbors = trial.suggest_int("n_neighbors", 2, 10) 
        weights = trial.suggest_categorical("weights", ["uniform", "distance"]) 
        p = trial.suggest_int("p", 1, 2)
        self.model = KNeighborsClassifier(n_neighbors = n_neighbors, weights = weights, p = p)   
      #LDA tuning
      if self.classifier_name == "LinearDiscriminantAnalysis":
        solver = trial.suggest_categorical("solver", ["svd", "lsqr", "eigen"])
        store_covariance = trial.suggest_categorical("store_covariance", [True, False])
        self.model = LinearDiscriminantAnalysis(solver = solver, store_covariance= store_covariance)
      # Logistic regression tuning
      if self.classifier_name == "LogisticRegression":
        C = trial.suggest_float("C", 0, 10)
        penalty = trial.suggest_categorical("penalty", ["l1", "l2"])
        self.model = LogisticRegression(C = C, penalty = penalty, solver = 'liblinear')
      #SVM tuning
      if self.classifier_name == "SVC":
        kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])
        C = trial.suggest_float("C", 0, 10)
        self.model = SVC(C = C, kernel= kernel)
      #Fit the  model and predict on the outer test sets after hyperparameter tuning in the inner cross validation loop
      cost_function = make_scorer(cohen_kappa_score)
      scores = cross_val_score(self.model, X_train, y_train, cv = self.inner_cv , scoring= cost_function)
      return scores.mean()
    
    ##################### Outer cross validation loop to evade overfitting on a single train/test split ############################
    # Number of of outer cross validation loops deterimined by the number of seeds for the trials
    trial_counter = 0
    for seed in self.trials_seeds:
      trial_scores = np.zeros((self.outer_folds, 6))
      base_trial_scores = np.zeros((self.outer_folds, 6))
      outer_cv_counter = 0
      # Create the stratified fold objects for the outer and inner cross validation loops with different random states per trial
      self.outer_cv = StratifiedKFold(n_splits = self.outer_folds, shuffle = self.shuffle, random_state= seed)
      self.inner_cv = StratifiedKFold(n_splits = self.inner_folds, shuffle = self.shuffle, random_state= seed)

      for train_index, test_index in self.outer_cv.split(X, y):
        #Outer cross validation loop to evade overfitting on a single train/test split
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # inner cross validation loop to tune hyperparameterss
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=50)
        params = study.best_params
      
        #Set the best hyperparameters for the model
        self.model.set_params(**params)
        parameters.append(params)

        #Fit the model and predict on the outer test sets after hyperparameter tuning in the inner cross validation loop
        self.model.fit(X_train, y_train)
        y_pred = self.model.predict(X_test)
        #Fit baseline classifier_name and predict on the outer test sets
        baseline_model.fit(X_train, y_train)
        y_base_pred = baseline_model.predict(X_test)

        #Calculate scores for each outer cross validation loop and trial
        trial_scores[outer_cv_counter,:] = self.score(y_test, y_pred)
        base_trial_scores[outer_cv_counter,:] = self.score(y_test, y_base_pred)
        # Move to the next outer cross validation fold
        outer_cv_counter += 1       
      #Increase the trial counter and go to the next trial
      scores_df.loc[f"Trial {trial_counter + 1}"] = trial_scores.mean(axis=0)
      baseline_df.loc[f"Trial {trial_counter + 1}"] = base_trial_scores.mean(axis=0)
      trial_counter += 1
    
    ######################### Results section ########################################
    if self.verbose:
      mean_var_df = pd.concat([scores_df.mean(axis=0),scores_df.var(axis=0)], axis=1).rename(columns={0:"Mean", 1:"Variance"})
      base_mean_var_df = pd.concat([baseline_df.mean(axis=0), baseline_df.var(axis=0)], axis=1).rename(columns={0:"Mean", 1:"Variance"})
      print(f"The average scores and their variance for {self.classifier_name} across all {self.trial_num} trials are:")
      print(mean_var_df,"\n")
      print (mean_var_df.mean(axis=0))
      print(f"The average Naive Bayes baseline scores and their variance across all {self.trial_num} trial are:")
      print(base_mean_var_df,"\n")
      print(base_mean_var_df.mean(axis=0))
    return scores_df, baseline_df, parameters 

  def score(self,y_test, y_pred):
    return roc_auc_score(y_test, y_pred), matthews_corrcoef(y_test, y_pred), balanced_accuracy_score(y_test, y_pred),\
    f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)

In [293]:
knn_tuner = BinaryClassifierNCV(classifier_name = "KNeighborsclassifier")
scores_df, baseline_scores_df, parameters = knn_tuner.fit(X, y)

The random seeds for each trial are: [102 435 860 270 106  71 700  20 614 121] 

The average scores and their variance for KNeighborsclassifier across all 10 trials are:
                       Mean  Variance
F1                 0.766834  0.000637
Balanced Accuracy  0.589937  0.002311
Precision          0.766834  0.000637
MCC                0.686463  0.001339
Recall             0.830384  0.001391
ROC AUC            0.597582  0.002022 

Mean        0.706339
Variance    0.001389
dtype: float64
The average Naive Bayes baseline scores and their variance across all 10 trial are:
                       Mean  Variance
F1                 0.843637  0.000116
Balanced Accuracy  0.717075  0.000545
Precision          0.843637  0.000116
MCC                0.796065  0.000220
Recall             0.865219  0.000537
ROC AUC            0.750659  0.000330 

Mean        0.802715
Variance    0.000311
dtype: float64


In [286]:
lda_tuner = BinaryClassifierNCV(classifier_name = "LinearDiscriminantAnalysis")
scores_df, baseline_scores_df, parameters = lda_tuner.fit(X, y)

The random seeds for each trial are: [102 435 860 270 106  71 700  20 614 121] 

The average scores and their variance for LinearDiscriminantAnalysis across all 10 trials are:
                       Mean  Variance
F1                 0.815423  0.000252
Balanced Accuracy  0.705657  0.000897
Precision          0.815423  0.000252
MCC                0.760076  0.000802
Recall             0.941380  0.000317
ROC AUC            0.651429  0.000877 

Mean        0.781565
Variance    0.000566
dtype: float64
The average Naive Bayes baseline scores and their variance across all 10 trial are:
                       Mean  Variance
F1                 0.843637  0.000116
Balanced Accuracy  0.717075  0.000545
Precision          0.843637  0.000116
MCC                0.796065  0.000220
Recall             0.865219  0.000537
ROC AUC            0.750659  0.000330 

Mean        0.802715
Variance    0.000311
dtype: float64


In [292]:
lr_tuner = BinaryClassifierNCV(classifier_name = "LogisticRegression")
scores_df,baseline_scores_df, parameters =  lr_tuner.fit(X, y)

The random seeds for each trial are: [102 435 860 270 106  71 700  20 614 121] 

The average scores and their variance for LogisticRegression across all 10 trials are:
                       Mean  Variance
F1                 0.881040  0.000338
Balanced Accuracy  0.789921  0.000899
Precision          0.881040  0.000338
MCC                0.847771  0.000536
Recall             0.910432  0.000610
ROC AUC            0.803297  0.001468 

Mean        0.852250
Variance    0.000698
dtype: float64
The average Naive Bayes baseline scores and their variance across all 10 trial are:
                       Mean  Variance
F1                 0.843637  0.000116
Balanced Accuracy  0.717075  0.000545
Precision          0.843637  0.000116
MCC                0.796065  0.000220
Recall             0.865219  0.000537
ROC AUC            0.750659  0.000330 

Mean        0.802715
Variance    0.000311
dtype: float64


In [276]:
svm_tuner = BinaryClassifierNCV(classifier_name = "SVC")
scores_df,baseline_scores_df, parameters  = svm_tuner.fit(X, y)

The random seeds for each trial are: [102 435 860 270 106  71 700  20 614 121] 

The average scores and their variance for SVC across all 10 trials are:
                       Mean  Variance
F1                 0.892429  0.000167
Balanced Accuracy  0.804552  0.000607
Precision          0.892429  0.000167
MCC                0.860839  0.000341
Recall             0.904973  0.000577
ROC AUC            0.830440  0.000500
The average Naive Bayes baseline scores and their variance across all 10 trial are:
                       Mean  Variance
F1                 0.843637  0.000116
Balanced Accuracy  0.717075  0.000545
Precision          0.843637  0.000116
MCC                0.796065  0.000220
Recall             0.865219  0.000537
ROC AUC            0.750659  0.000330
